In [866]:
from collections import defaultdict
import re
import pandas as pd

In [867]:
with open('inputs/input20_test.txt') as f:
    example_tiles = f.read().strip().split('\n\n')

with open('inputs/input20.txt') as f:
    real_tiles = f.read().strip().split('\n\n')
    
len(real_tiles)

144

In [868]:
# returns a list of 2 dicts: edge adjacencies, and actual tile contents
def make_tile_dicts(tiles):
    tile_edge_dict = {}
    tile_data_dict = {}
    for tile in tiles:
        tile_lines = tile.split('\n')
        tile_id = re.search('[0-9]+', tile_lines[0]).group()
        tile_data = pd.DataFrame([list(x) for x in tile_lines[1:]])
        tile_data_dict[tile_id] = tile_data
        edge_1 = ''.join(list(tile_data.iloc[0]))  # top
        edge_2 = ''.join(list(tile_data.iloc[:,-1:].values.flatten()))  # right
        edge_3 = ''.join(list(tile_data.iloc[-1]))  # bottom
        edge_4 = ''.join(list(tile_data.iloc[:,0]))  # left
        tile_edge_dict[tile_id] = {1: edge_1, 2: edge_2, 3: edge_3, 4: edge_4}
    return tile_edge_dict, tile_data_dict

tile_edge_dict, tile_data_dict = make_tile_dicts(example_tiles)
tile_edge_dict

{'2311': {1: '..##.#..#.', 2: '...#.##..#', 3: '..###..###', 4: '.#####..#.'},
 '1951': {1: '#.##...##.', 2: '.#####..#.', 3: '#...##.#..', 4: '##.#..#..#'},
 '1171': {1: '####...##.', 2: '.#..#.....', 3: '.....##...', 4: '###....##.'},
 '1427': {1: '###.##.#..', 2: '..###.#.#.', 3: '..##.#..#.', 4: '#..#......'},
 '1489': {1: '##.#.#....', 2: '.....#..#.', 3: '###.##.#..', 4: '#...##.#.#'},
 '2473': {1: '#....####.', 2: '...###.#..', 3: '..###.#.#.', 4: '####...##.'},
 '2971': {1: '..#.#....#', 2: '#...##.#.#', 3: '...#.#.#.#', 4: '.###..#...'},
 '2729': {1: '...#.#.#.#', 2: '#..#......', 3: '#.##...##.', 4: '.#....####'},
 '3079': {1: '#.#.#####.', 2: '.#....#...', 3: '..#.###...', 4: '#..##.#...'}}

In [869]:
edge_matches = defaultdict(dict)  # tile ID -> {which edge -> matching tile ID}
for id1, edge_dict1 in tile_edge_dict.items():
    for which_edge1, content1 in edge_dict1.items():
        # how many other edges does this match?
        
        match_count = 0
        for id2, edge_dict2 in tile_edge_dict.items():
            if id1 == id2:  # don't match self
                continue
            for which_edge2, content2 in edge_dict2.items():
                if content1 in [content2, content2[::-1]]:
                    match_count += 1
                    edge_matches[id1][which_edge1] = id2
        if match_count > 2:
            raise ValueError('what now')

In [870]:
# get all corner tiles
corner_tiles = {x: y for x, y in edge_matches.items() if len(y) == 2}

# get all the edge tiles
edge_tiles = {x: y for x, y in edge_matches.items() if len(y) == 3}

# get all the edge tiles
inner_tiles = {x: y for x, y in edge_matches.items() if len(y) == 4}

In [871]:
# put it all together

n = int(math.sqrt(len(tile_edge_dict)))
image = []
for i in range(n):
    image.append([''] * n)
    
placed_tiles = set()
top_left_corner_tile = list(corner_tiles.keys())[0]
image[0][0] = top_left_corner_tile  # take any corner, put it in the top left
placed_tiles.add(top_left_corner_tile)

# modifies: image, placed_tiles
def check_tile_dicts(tile_dicts, known_pos, pos_to_check):
    for tile_id, edge_dict in tile_dicts.items():
        if image[known_pos[0]][known_pos[1]] in edge_dict.values() and tile_id not in placed_tiles:
            image[pos_to_check[0]][pos_to_check[1]] = tile_id
            placed_tiles.add(tile_id)
            return True
    return False
        
# left edge
for i in range(1, n):
    # check edge tiles first, then corner tiles
    if not check_tile_dicts(edge_tiles, (i-1, 0), (i, 0)):
        check_tile_dicts(corner_tiles, (i-1, 0), (i, 0))
            
# top edge
for j in range(1, n):
    if not check_tile_dicts(edge_tiles, (0, j-1), (0, j)):
        check_tile_dicts(corner_tiles, (0, j-1), (0, j))

# right edge
for i in range(1, n):
    if not check_tile_dicts(edge_tiles, (i-1, n-1), (i, n-1)):
        check_tile_dicts(corner_tiles, (i-1, n-1), (i, n-1))

# bottom edge
for j in range(1, n):
    if not check_tile_dicts(edge_tiles, (n-1, j-1), (n-1, j)):
        check_tile_dicts(corner_tiles, (n-1, j-1), (n-1, j))
            
# figure out which inner tiles are what
for i in range(1, n-1):
    for j in range(1, n-1):
        upper_neighbor = image[i-1][j]
        left_neighbor = image[i][j-1]
        for tile_id, edge_dict in inner_tiles.items():
            if upper_neighbor in edge_dict.values() and left_neighbor in edge_dict.values() and tile_id not in placed_tiles:
                image[i][j] = tile_id
                placed_tiles.add(tile_id)
                break

show(image)

['1951', '2729', '2971']
['2311', '1427', '1489']
['3079', '2473', '1171']


In [872]:
def rotate_clockwise(df):
    t = df.T
    return t[t.columns[::-1]]

def vertical_flip(df):
    return df[::-1]

def horizontal_flip(df):
    return df[df.columns[::-1]]

def is_top_fit(top, bottom):
    bottom_row_of_top = top.iloc[-1]
    top_row_of_bottom = bottom.iloc[0]
    return list(top_row_of_bottom) == list(bottom_row_of_top)

def is_left_fit(left, right):
    right_col_of_left = left.iloc[:,-1:]
    left_col_of_right = right.iloc[:,0]
    return list(left_col_of_right) == list(right_col_of_left.values.flatten())

In [873]:
# given 2 tiles and the needed orientation, transform them both so that it works

def make_all_transforms(df):
    df_rotated90 = rotate_clockwise(df)
    df_rotated180 = rotate_clockwise(df_rotated90)
    df_rotated270 = rotate_clockwise(df_rotated180)
    df_horizontal_flipped = horizontal_flip(df)
    df_flipped = vertical_flip(df)
    df_flipped_rotated90 = rotate_clockwise(df_flipped)
    df_flipped_rotated270 = rotate_clockwise(rotate_clockwise(df_flipped_rotated90))
    df_transforms = [df, df_rotated90, df_rotated180, df_rotated270, df_horizontal_flipped,
                     df_flipped, df_flipped_rotated90, df_flipped_rotated270]
    return df_transforms

def transform_top_bottom_tiles(top, bottom):
    # try all configurations of top and bottom until is_top_fit(top, bottom) returns True, then return
    # transformed top and bottom tiles
    for top_transform in make_all_transforms(top):
        for bottom_transform in make_all_transforms(bottom):
            if is_top_fit(top_transform, bottom_transform):
                return top_transform, bottom_transform

def transform_left_right_tiles(left, right):
    for left_transform in make_all_transforms(left):
        for right_transform in make_all_transforms(right):
            if is_left_fit(left_transform, right_transform):
                return left_transform, right_transform
            
def transform_right_tile(left, right):
    for right_transform in make_all_transforms(right):
        if is_left_fit(left, right_transform):
            return right_transform
    print('Could not transform right tile. Left, right:')
    print(left)
    print(right)
    raise ValueError
        
def transform_bottom_tile(top, bottom):
    for bottom_transform in make_all_transforms(bottom):
        if is_top_fit(top, bottom_transform):
            return bottom_transform
    print('Could not transform bottom tile. Top, bottom:')
    print(top)
    print(bottom)
    raise ValueError
    

In [874]:
def make_df_from_actual_image(actual_image):
    concat_list = []
    for row_list in actual_image:
        row_concat = []
        for df in row_list:
            df.index = range(len(df))  # otherwise concat tries to be too smart
            row_concat.append(df)
        row_concat_df = pd.concat(row_concat, axis=1)
        concat_list.append(row_concat_df)

    for df in concat_list:
        df.columns = list(range(len(df.columns)))

    return pd.concat(concat_list)

In [875]:
# initialize actual image array
n = int(math.sqrt(len(tile_edge_dict)))
actual_image = []  # array of arrays of DataFrames
for i in range(n):
    actual_image.append([None] * n)
    
for i in range(n):
    for j in range(n):
        actual_image[i][j] = tile_data_dict[image[i][j]]

# fix the 3 tiles in the top left corner to give the whole thing an orientation
def fix_top_left_corner(top, right, bottom):
    for top_transform in make_all_transforms(top):
        for right_transform in make_all_transforms(right):
            for bottom_transform in make_all_transforms(bottom):
                if is_left_fit(top_transform, right_transform) and is_top_fit(top_transform, bottom_transform):
                    return (top_transform, right_transform, bottom_transform)
                
top = actual_image[0][0]
right = actual_image[0][1]
bottom = actual_image[1][0]

actual_image[0][0], actual_image[0][1], actual_image[1][0] = fix_top_left_corner(top, right, bottom)

# fix the top row
i = 0
for j in range(1, n):
    left, right = actual_image[i][j-1], actual_image[i][j]
    actual_image[i][j] = transform_right_tile(left, right)
    
# fix left column
j = 0
for i in range(1, n):
    top, bottom = actual_image[i-1][j], actual_image[i][j]
    actual_image[i][j] = transform_bottom_tile(top, bottom)

# fix inner ones starting with (1, 1) and fanning out: should work by aligning to top at this point
for i in range(1, n):
    for j in range(1, n):
        top, bottom = actual_image[i-1][j], actual_image[i][j]
        actual_image[i][j] = transform_bottom_tile(top, bottom)
        
pd.set_option("display.max_columns", 100)
make_df_from_actual_image(actual_image)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,#,.,.,#,.,.,#,.,#,#,#,#,#,#,.,.,.,.,#,.,.,.,.,#,.,.,#,#,#,.
1,.,.,#,#,#,#,.,.,.,.,.,#,#,#,#,#,.,.,#,.,.,.,.,.,#,#,#,.,.,.
2,.,#,#,#,#,#,.,.,#,#,#,.,.,#,.,#,.,#,#,.,.,#,#,.,.,#,.,#,.,#
3,.,.,#,.,#,.,.,.,#,#,#,.,#,#,#,.,.,.,#,#,#,.,#,#,.,#,#,.,.,.
4,#,#,.,#,.,#,#,.,#,.,.,#,#,.,#,.,#,#,.,.,.,#,#,.,#,#,#,#,#,#
5,#,.,.,#,#,.,#,#,#,.,.,.,#,#,#,#,.,.,#,#,#,.,#,#,#,#,.,#,#,.
6,.,.,.,.,#,.,#,.,.,.,.,#,#,.,#,#,.,.,.,.,.,#,.,.,#,.,.,#,#,.
7,#,#,.,#,#,.,#,.,.,#,#,#,.,#,.,.,#,.,.,#,#,#,#,.,#,.,#,.,.,.
8,.,.,#,#,#,.,#,#,.,#,#,.,.,.,.,#,.,.,.,.,.,#,#,#,.,#,.,.,.,.
9,.,#,.,.,#,#,#,#,#,.,.,.,.,.,.,.,#,.,.,#,#,.,#,.,#,#,.,.,.,#


In [876]:
# this checks out. Okay now do this same thing but remove the borders first:

def make_borderless_df_from_actual_image(actual_image):
    concat_list = []
    for row_list in actual_image:
        row_concat = []
        for df in row_list:
            df_copy = df.copy()  # avoid changing the actual image as the source of truth
            # set row and column names to be ranges otherwise concat tries to be too smart
            df_copy.index = range(len(df_copy))
            df_copy.columns = range(len(df_copy))
            borderless_df = df_copy.head(-1).tail(-1).drop(
                df_copy.columns[-1], axis=1).drop(
                df_copy.columns[0], axis=1)
            
            row_concat.append(borderless_df)
        row_concat_df = pd.concat(row_concat, axis=1)
        concat_list.append(row_concat_df)

    for df in concat_list:
        df.columns = list(range(len(df.columns)))

    return pd.concat(concat_list)

In [877]:
borderless_df = make_borderless_df_from_actual_image(actual_image)

In [878]:
def stringify(df):
    return df.to_string(header=False, index=False).replace(' ', '')

print(stringify(borderless_df))

.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.##..
#.#.##.###.#.##.##.#####
..##.###.####..#.####.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.######.
.###.###.#######..#####.
..##.#..#..#.#######.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#


In [879]:
sea_monster = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""[1:-1]

sea_monster_df = pd.DataFrame([list(x) for x in sea_monster.split('\n')])
monster_height, monster_width = sea_monster_df.shape
sea_monster_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,,,,,,#,
1,#,,,,,#,#,,,,,#,#,,,,,#,#,#
2,,#,,,#,,,#,,,#,,,#,,,#,,,


In [880]:
def binarize(monster_cell):
    return monster_cell == '#'

binary_sea_monster = sea_monster_df.applymap(binarize)
binary_sea_monster

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,False,False,True,True,True
2,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False


In [881]:
def num_hits(binary_df):
    return sum(binary_df.sum())

def is_match(sea_monster_df, subset_df):
    monster_height, monster_width = sea_monster_df.shape
    binary_sea_monster = sea_monster_df.applymap(binarize)
    sea_monster_sum = num_hits(binary_sea_monster)
    
    binary_subset = subset_df.applymap(binarize)
    binary_subset.columns = list(range(monster_width))
    binary_subset.index = list(range(monster_height))
    assert sea_monster_df.shape == subset_df.shape
    
    return num_hits(binary_subset & binary_sea_monster) == sea_monster_sum


In [882]:
# move a sliding window through borderless_df and look for monsters

num_monsters = 0
for i in range(len(borderless_df) - monster_height):
    for j in range(len(borderless_df) - monster_width):
        borderless_df_subset = borderless_df.iloc[i:i+monster_height, j:j+monster_width]
        if is_match(sea_monster_df, borderless_df_subset):
            print('monster!! at %d %d' % (i, j))
            num_monsters += 1

monster!! at 2 2
monster!! at 16 1


In [883]:
# num_monsters * num_hits(sea_monster_df)
num_monster_tiles = num_monsters * num_hits(binary_sea_monster)
binary_borderless_df = borderless_df.applymap(binarize)
num_all_pound_tiles = num_hits(binary_borderless_df)
num_all_pound_tiles - num_monster_tiles

273

# Real input

In [884]:
tile_edge_dict, tile_data_dict = make_tile_dicts(real_tiles)

edge_matches = defaultdict(dict)  # tile ID -> {which edge -> matching tile ID}
for id1, edge_dict1 in tile_edge_dict.items():
    for which_edge1, content1 in edge_dict1.items():
        # how many other edges does this match?
        
        match_count = 0
        for id2, edge_dict2 in tile_edge_dict.items():
            if id1 == id2:  # don't match self
                continue
            for which_edge2, content2 in edge_dict2.items():
                if content1 in [content2, content2[::-1]]:
                    match_count += 1
                    edge_matches[id1][which_edge1] = id2
        if match_count > 2:
            raise ValueError('what now')

In [885]:
# get all corner tiles
corner_tiles = {x: y for x, y in edge_matches.items() if len(y) == 2}

# get all the edge tiles
edge_tiles = {x: y for x, y in edge_matches.items() if len(y) == 3}

# get all the edge tiles
inner_tiles = {x: y for x, y in edge_matches.items() if len(y) == 4}

In [886]:
corner_tiles

{'2243': {3: '2663', 4: '2707'},
 '1213': {1: '3259', 2: '3511'},
 '2953': {3: '3299', 4: '2371'},
 '2273': {2: '1543', 3: '2617'}}

In [887]:
import numpy as np
np.product(list(map(int, corner_tiles.keys())))  # answer for puzzle 1

18262194216271

In [888]:
# put it all together

n = int(math.sqrt(len(tile_edge_dict)))
image = []
for i in range(n):
    image.append([''] * n)
    
placed_tiles = set()
top_left_corner_tile = list(corner_tiles.keys())[0]
image[0][0] = top_left_corner_tile  # take any corner, put it in the top left
placed_tiles.add(top_left_corner_tile)

# modifies: image, placed_tiles
def check_tile_dicts(tile_dicts, known_pos, pos_to_check):
    for tile_id, edge_dict in tile_dicts.items():
        if image[known_pos[0]][known_pos[1]] in edge_dict.values() and tile_id not in placed_tiles:
            image[pos_to_check[0]][pos_to_check[1]] = tile_id
            placed_tiles.add(tile_id)
            return True
    return False
        
# left edge
for i in range(1, n):
    # check edge tiles first, then corner tiles
    if not check_tile_dicts(edge_tiles, (i-1, 0), (i, 0)):
        check_tile_dicts(corner_tiles, (i-1, 0), (i, 0))
            
# top edge
for j in range(1, n):
    if not check_tile_dicts(edge_tiles, (0, j-1), (0, j)):
        check_tile_dicts(corner_tiles, (0, j-1), (0, j))

# right edge
for i in range(1, n):
    if not check_tile_dicts(edge_tiles, (i-1, n-1), (i, n-1)):
        check_tile_dicts(corner_tiles, (i-1, n-1), (i, n-1))

# bottom edge
for j in range(1, n):
    if not check_tile_dicts(edge_tiles, (n-1, j-1), (n-1, j)):
        check_tile_dicts(corner_tiles, (n-1, j-1), (n-1, j))
            
# figure out which inner tiles are what
for i in range(1, n-1):
    for j in range(1, n-1):
        upper_neighbor = image[i-1][j]
        left_neighbor = image[i][j-1]
        for tile_id, edge_dict in inner_tiles.items():
            if upper_neighbor in edge_dict.values() and left_neighbor in edge_dict.values() and tile_id not in placed_tiles:
                image[i][j] = tile_id
                placed_tiles.add(tile_id)
                break

show(image)

['2243', '2707', '1481', '3559', '1489', '1583', '1307', '2549', '2677', '1289', '3511', '1213']
['2663', '1549', '3821', '2161', '2003', '1471', '3769', '2207', '1933', '3329', '3187', '3259']
['2087', '1409', '1399', '3833', '1873', '1601', '1801', '1019', '3853', '2237', '2099', '1531']
['3067', '3343', '2477', '2609', '1831', '1063', '1097', '3631', '2551', '2591', '1787', '1867']
['3881', '3491', '2753', '3557', '1657', '3923', '3643', '1597', '1259', '1931', '1979', '2441']
['2333', '3467', '2063', '2843', '1447', '1879', '3209', '3767', '3541', '2357', '2711', '1621']
['1607', '2411', '1009', '3037', '1733', '3313', '1181', '3251', '2749', '3929', '3023', '1667']
['2111', '2909', '1187', '3539', '2957', '2287', '3863', '1993', '1229', '1571', '2699', '1759']
['3877', '1321', '3011', '2281', '2903', '1637', '1087', '1049', '2579', '3229', '3203', '3041']
['1117', '1913', '3469', '3571', '3533', '1907', '1663', '3407', '3391', '2297', '3623', '2143']
['2371', '2341', '2467', '2621

In [889]:
# initialize actual image array

def make_actual_image(tile_edge_dict, image):
    n = int(math.sqrt(len(tile_edge_dict)))
    actual_image = []  # array of arrays of DataFrames
    for i in range(n):
        actual_image.append([None] * n)

    for i in range(n):
        for j in range(n):
            actual_image[i][j] = tile_data_dict[image[i][j]]
    return actual_image

# fix the 3 tiles in the top left corner to give the whole thing an orientation
def fix_top_left_corner(top, right, bottom):
    for top_transform in make_all_transforms(top):
        for right_transform in make_all_transforms(right):
            for bottom_transform in make_all_transforms(bottom):
                if is_left_fit(top_transform, right_transform) and is_top_fit(top_transform, bottom_transform):
                    return (top_transform, right_transform, bottom_transform)
                
actual_image = make_actual_image(tile_edge_dict, image)
top = actual_image[0][0]
right = actual_image[0][1]
bottom = actual_image[1][0]

try:
    top_fixed, right_fixed, bottom_fixed = fix_top_left_corner(top, right, bottom)
except TypeError:
    image.reverse()
    actual_image = make_actual_image(tile_edge_dict, image)
    top = actual_image[0][0]
    right = actual_image[0][1]
    bottom = actual_image[1][0]
    top_fixed, right_fixed, bottom_fixed = fix_top_left_corner(top, right, bottom)
    
actual_image[0][0] = top_fixed
actual_image[0][1] = right_fixed
actual_image[1][0] = bottom_fixed

# fix the top row
i = 0
for j in range(1, n):
    left, right = actual_image[i][j-1], actual_image[i][j]
    actual_image[i][j] = transform_right_tile(left, right)
    
# fix left column
j = 0
for i in range(1, n):
    top, bottom = actual_image[i-1][j], actual_image[i][j]
    try:
        actual_image[i][j] = transform_bottom_tile(top, bottom)
    except AttributeError:
        raise ValueError('check %d, %d' % (i, j))

# fix inner ones starting with (1, 1) and fanning out: should work by aligning to top at this point
for i in range(1, n):
    for j in range(1, n):
        try:
            top, bottom = actual_image[i-1][j], actual_image[i][j]
            actual_image[i][j] = transform_bottom_tile(top, bottom)
        except AttributeError:
            raise ValueError('check %d, %d' % (i, j))

In [890]:
pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 120)
make_df_from_actual_image(actual_image)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,#,#,.,.,#,#,.,#,.,.,.,.,.,#,.,.,.,.,#,#,#,#,#,.,.,#,.,#,.,.,.,.,#,#,.,#,.,#,.,#,#,.,#,#,.,.,#,#,#,.,.,.,#,.,#,.,#,.,#,#,#,.,.,.,#,#,#,#,#,#,#,.,.,.,#,#,.,#,#,.,.,#,#,.,#,.,#,#,.,.,.,#,.,.,.,.,.,.,.,.,.,#,#,#,#,#,.,#,#,.,.,#,.,#,.,.,#,#,#,#
1,#,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,#,#,.,#,#,#,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,#,#,.,#,#,.,#,#,.,.,.,.,#,.,.,#,.,.,.,#,#,#,.,#,#,.,.,.,#,.,.,.,.,#,#,.,.,.,.,.,.,.,.,.,.,#,#,.,#,.,.,#,.,#,#,.,#,.,#,.,#,.,.,#,#,#,.,.,.,.,.,.,.,#
2,.,#,.,#,#,.,.,.,.,#,#,#,#,.,#,.,.,.,.,.,.,.,.,#,.,.,.,.,.,.,.,.,#,.,.,#,.,.,#,#,#,.,.,.,.,.,.,.,.,#,#,#,.,.,#,.,.,#,.,.,.,.,.,#,#,#,.,.,.,#,#,#,.,.,.,.,.,.,.,#,#,#,.,#,.,.,.,#,.,.,.,#,#,.,.,.,.,#,.,#,#,.,.,.,.,.,.,.,.,#,#,#,.,.,#,#,.,.,.,#
3,#,#,.,.,.,.,#,.,#,.,.,.,.,#,#,.,#,.,#,.,.,.,.,.,.,#,.,.,.,.,.,.,#,.,.,.,.,.,.,.,.,#,.,.,.,.,.,.,.,#,#,#,.,.,#,.,#,#,.,#,#,#,.,.,#,.,#,#,.,#,#,#,.,#,#,#,#,#,#,.,.,.,#,#,#,.,.,#,.,.,.,.,.,.,.,#,#,.,.,.,.,.,.,.,.,.,.,.,#,#,#,.,.,.,.,.,.,.,.,.
4,.,.,.,.,#,.,.,.,#,#,#,.,#,.,.,#,#,#,#,#,#,.,#,.,#,#,.,.,.,.,.,.,#,.,.,.,.,#,#,#,#,.,.,#,.,#,#,.,.,.,.,#,.,#,#,#,#,.,.,#,#,#,#,.,.,#,#,.,#,#,#,.,.,#,.,#,#,.,.,#,#,#,#,.,.,.,.,.,#,#,#,.,.,#,.,.,.,.,.,#,#,.,.,.,.,.,#,#,.,.,.,.,.,.,#,.,.,.,.,.
5,#,#,.,#,.,.,.,.,#,.,.,.,.,.,.,.,.,.,#,#,#,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,#,.,.,.,.,#,#,#,#,.,#,#,.,.,.,.,#,#,.,.,#,.,.,.,.,.,#,.,.,.,.,.,.,#,#,#,.,.,#,.,.,.,.,#,#,.,.,.,.,.,.,#,.,.,.,.,.,.,.,.,#,#,.,#,#,.,.,.,.,.,.,.,#,#,#,.,.,.,.,#,#,.,#,#
6,.,.,#,.,#,.,.,.,.,#,#,.,#,.,#,.,#,#,.,#,#,.,#,.,.,.,.,.,#,#,#,.,.,.,.,.,.,.,.,#,#,#,.,#,#,.,.,.,.,#,#,.,#,#,.,#,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,#,.,.,#,#,#,.,#,#,.,.,#,.,#,.,#,.,.,.,.,.,.,.,.,.,#,.,#,#,.,.,.,.,#,.,#,.,.,.,.,.,#,.,.,.,#,.,.
7,.,.,.,.,#,.,.,#,.,#,#,.,.,.,.,#,.,#,.,.,.,#,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,.,#,.,#,.,.,.,#,.,.,.,.,.,#,.,.,.,#,#,#,#,.,.,.,.,#,.,#,.,#,#,.,#,#,#,.,.,.,#,#,#,#,.,.,.,.,#,#,#,.,.,.,.,#,.,.,#,.,.,#,#,.,#,#,#,.,.,.,#,#,#,.,.,#,.,.,#,#,.,#
8,.,.,#,.,.,.,#,.,.,#,#,.,.,.,.,#,.,.,.,#,#,.,.,#,#,.,.,.,#,.,.,.,.,#,.,#,.,.,.,#,#,.,.,#,#,.,#,.,.,.,.,.,.,#,.,.,#,.,.,#,#,.,.,.,.,.,#,#,.,.,.,#,#,.,#,.,.,#,.,.,.,.,#,#,.,#,#,.,.,#,#,#,.,#,.,#,#,.,.,.,.,.,.,#,.,#,.,.,.,#,#,.,.,.,.,.,.,.,.,.
9,.,#,#,#,#,#,.,.,.,.,.,.,.,.,.,.,.,#,.,#,#,.,.,.,#,#,.,#,.,.,.,#,.,#,.,#,.,#,.,#,#,.,#,#,.,#,.,.,#,.,.,.,#,#,.,#,.,.,#,#,#,#,#,#,.,.,.,#,#,.,.,.,#,#,#,#,#,.,.,#,#,#,.,#,#,.,#,.,.,.,.,#,.,.,#,.,.,#,#,.,.,.,.,.,#,#,.,#,.,#,#,#,#,#,.,#,.,.,.,#


In [891]:
# find some monsters

borderless_df = make_borderless_df_from_actual_image(actual_image)

sea_monster = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""[1:-1]

sea_monster_df = pd.DataFrame([list(x) for x in sea_monster.split('\n')])
monster_height, monster_width = sea_monster_df.shape

def binarize(monster_cell):
    return monster_cell == '#'

def num_hits(binary_df):
    return sum(binary_df.sum())

def is_match(sea_monster_df, subset_df):
    monster_height, monster_width = sea_monster_df.shape
    binary_sea_monster = sea_monster_df.applymap(binarize)
    sea_monster_sum = num_hits(binary_sea_monster)
    
    binary_subset = subset_df.applymap(binarize)
    binary_subset.columns = list(range(monster_width))
    binary_subset.index = list(range(monster_height))
    assert sea_monster_df.shape == subset_df.shape
    
    return num_hits(binary_subset & binary_sea_monster) == sea_monster_sum

# move a sliding window through borderless_df and look for monsters

num_monsters = 0
for i in range(len(borderless_df) - monster_height):
    for j in range(len(borderless_df) - monster_width):
        borderless_df_subset = borderless_df.iloc[i:i+monster_height, j:j+monster_width]
        if is_match(sea_monster_df, borderless_df_subset):
            print('monster!! at %d %d' % (i, j))
            num_monsters += 1

binary_sea_monster = sea_monster_df.applymap(binarize)
num_monster_tiles = num_monsters * num_hits(binary_sea_monster)
binary_borderless_df = borderless_df.applymap(binarize)
num_all_pound_tiles = num_hits(binary_borderless_df)
num_all_pound_tiles - num_monster_tiles

monster!! at 1 48
monster!! at 2 25
monster!! at 5 52
monster!! at 7 16
monster!! at 9 74
monster!! at 10 50
monster!! at 13 22
monster!! at 18 49
monster!! at 19 26
monster!! at 19 73
monster!! at 21 1
monster!! at 24 67
monster!! at 25 8
monster!! at 25 31
monster!! at 29 69
monster!! at 30 28
monster!! at 32 3
monster!! at 37 10
monster!! at 37 41
monster!! at 38 71
monster!! at 41 17
monster!! at 45 55
monster!! at 49 27
monster!! at 50 3
monster!! at 54 64
monster!! at 55 31
monster!! at 59 6
monster!! at 60 47
monster!! at 63 19
monster!! at 64 73
monster!! at 69 15
monster!! at 69 49
monster!! at 73 57
monster!! at 75 15
monster!! at 78 53
monster!! at 79 22
monster!! at 84 7
monster!! at 85 36
monster!! at 86 73
monster!! at 90 23
monster!! at 91 58


2023